In [17]:
import os
import numpy as np
import csv
import torch
import sys
from datetime import datetime

csv.field_size_limit(sys.maxsize)

9223372036854775807

In [7]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [8]:
def readCSV(fname):
    file = open(f"{os.getcwd()}/{fname}.csv")
    csvreader = csv.reader(file)
    header = []
    header = next(csvreader)
    
    rows = []
    for row in csvreader:
        rows.append(row)
    file.close()
        
    return header, rows

In [11]:
coll_head, coll_rows = readCSV("Cleaning_collections")
api_head, api_rows = readCSV("Data_API")

In [13]:
api_rows[0]

['',
 '87094722.0',
 '0xed9ff1c311a611e5c457ce0a49bbbb7ed55efca988421aee27a01a8f8f6810f2',
 '0x76481caa104b5f6bccb540dae4cefaf1c398ebea',
 '',
 '0xe0fb7622091e3d9ef9b438471b10b9ea88c7cf6b',
 '',
 'https://card.godsunchained.com/?id=33&q=4',
 '',
 '',
 '',
 '0.0002',
 'ETH',
 '0.030318',
 '',
 '',
 'Godsunchained',
 'Godsunchained',
 '2019-11-30 00:00:00',
 '2019-11-30 23:59:33',
 '',
 "('Godsunchained', '87094722.0')",
 'Godsunchained',
 'Games']

In [14]:
api_head

['Smart_contract',
 'ID_token',
 'Transaction_hash',
 'Seller_address',
 'Seller_username',
 'Buyer_address',
 'Buyer_username',
 'Image_url_1',
 'Image_url_2',
 'Image_url_3',
 'Image_url_4',
 'Price_Crypto',
 'Crypto',
 'Price_USD',
 'Name',
 'Description',
 'Collection',
 'Market',
 'Datetime_updated',
 'Datetime_updated_seconds',
 'Permanent_link',
 'Unique_id_collection',
 'Collection_cleaned',
 'Category']

In [22]:
el = api_rows[53]
d1 = datetime.fromisoformat(el[18]).timestamp()
d2 = datetime.fromisoformat(el[19])
coll = el[16]

In [24]:
coll

'Godsunchained'

In [ ]:
for 